In [2]:
import torch
from pathlib import Path
from PIL import Image, ImageDraw, ImageEnhance, ImageOps
import pytesseract
import os
import random

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

model = torch.hub.load('ultralytics/yolov5', 'custom', path='D:/test3/exp4/weights/best.pt')

test_image_folder = 'D:/test3/container_dataset/test/images'
results_folder = 'D:/test3/ocr_results'
recognized_images_folder = 'D:/test3/ocr_results/recognized'
preprocessed_images_folder = 'D:/test3/ocr_results/preprocessed'
os.makedirs(results_folder, exist_ok=True)
os.makedirs(recognized_images_folder, exist_ok=True)
os.makedirs(preprocessed_images_folder, exist_ok=True)

correct_numbers_folder = 'D:/test3/picture_test_sataset/picture_test_sataset'
correct_numbers = {file_name: file_name.split('.')[0] for file_name in os.listdir(correct_numbers_folder)}

correct_count = 0
total_count = 0

def preprocess_image(img):
    # 增加對比度
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2)
    
    # 轉換為灰度圖像
    img = ImageOps.grayscale(img)
    
    # 二值化處理
    img = img.point(lambda x: 0 if x < 128 else 255, '1')
    
    return img

preprocessed_images = []
recognized_image_paths = []

for image_file in os.listdir(test_image_folder):
    if image_file.endswith(".jpg") or image_file.endswith(".png"):
        image_path = os.path.join(test_image_folder, image_file)
        image = Image.open(image_path)
        
        # 使用YOLOv5模型進行物件偵測
        results = model(image_path)
        detected_objects = results.xyxy[0].cpu().numpy()  # 獲取偵測結果並轉換為NumPy陣列
        
        # 繪製檢測結果
        draw = ImageDraw.Draw(image)
        for obj in detected_objects:
            x1, y1, x2, y2, conf, cls = obj
            draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        
        image.save(os.path.join(results_folder, f"{image_file}_detected.jpg"))
        
        for obj in detected_objects:
            x1, y1, x2, y2, conf, cls = obj
            cropped_img = image.crop((x1, y1, x2, y2))
            
            # 預處理裁剪出的圖片
            preprocessed_img = preprocess_image(cropped_img)
            preprocessed_img_path = os.path.join(preprocessed_images_folder, f"{image_file}_preprocessed_{len(preprocessed_images)}.jpg")
            preprocessed_img.save(preprocessed_img_path)
            preprocessed_images.append(preprocessed_img_path)
            
            # 使用Tesseract進行文字辨識
            recognized_text = pytesseract.image_to_string(preprocessed_img)
            recognized_text = ''.join(filter(str.isalnum, recognized_text))
            
            # 比較前11位數字
            correct_number = correct_numbers.get(image_file, "")
            if recognized_text[:11] == correct_number[:11]:
                correct_count += 1
                recognized_image_path = os.path.join(recognized_images_folder, f"{image_file}_recognized.jpg")
                image.save(recognized_image_path)
                recognized_image_paths.append(recognized_image_path)
            total_count += 1

accuracy = correct_count / total_count if total_count > 0 else 0
print(f"OCR 準確率: {accuracy:.2f}")

print("成功辨識的圖片已保存至資料夾:")
for img_path in recognized_image_paths:
    print(img_path)

print("隨機顯示5張預處理過的圖片:")
for img_path in random.sample(preprocessed_images, min(5, len(preprocessed_images))):
    print(img_path)

Using cache found in C:\Users\brain/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-25 Python-3.11.1 torch-2.1.0.dev20230307+cu118 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)

Fusing layers... 


WARNING ⚠️ torchvision==0.15 is incompatible with torch==2.1.
Run 'pip install torchvision==0.16' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation


Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


OCR 準確率: 0.82
成功辨識的圖片已保存至資料夾:
D:/test3/ocr_results/recognized\image_0001.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0002.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0007.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0008.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0009.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0010.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0011.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0012.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0014.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0018.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0019.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0020.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0021.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0022.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_0023.jpg_recognized.jpg
D:/test3/ocr_results/recognized\image_00